In [1]:
# SR with Importance Sampling Analysis
# This notebook follows the exact structure of the original notebooks

using Random
using Statistics
using Distributions
using DataFrames
using DataFramesMeta
using LinearAlgebra
using MixedModels
using JLD2

# Load local packages
using RLSR  # Now this works because it's in dependencies/
# using EM  # Uncomment if you have EM

In [2]:
# Set up paths
base_dir = ".."
results_dir = "$(base_dir)/results"
output_dir = "$(base_dir)/results/regression"

# Include additional code files
include("$(base_dir)/code/regression_sim.jl")  # Make sure to copy this from original
# include("$(base_dir)/code/sailing_base_mbsr_blockwise_betas_twotd.jl")  # If needed

# Now run analyses exactly like in the notebook
println("Running SR-IS Analysis")

Running SR-IS Analysis


In [3]:
# 7. Main analysis function
function run_sr_is_analysis()
    nsims = 50  # Number of simulated subjects per model
    
    # Parameters
    βSR = 2.0
    βMB = 2.0
    βBoat = 2.0
    island_bias = 0.0
    boat_bias = 0.0
    αHome = 0.5
    αAway = 0.5
    αM = 0.5
    γ = 0.95
    
    println("Running simulations...")
    
    # Pure MB
    println("  Pure MB...")
    df_mb_list = []
    for s in 1:nsims
        df = sim_blockwise_TD0TD1SRMB_subject(0.0, 0.0, 0.0, βMB, βBoat, island_bias, boat_bias, αHome, αAway, αM, γ, s)
        df[!, "model_type"] = "Pure MB"
        df[!, "subject"] = string("MB_", lpad(s, 3, "0"))
        push!(df_mb_list, df)
    end
    df_mb = vcat(df_mb_list...)
    
    # Pure SR
    println("  Pure SR...")
    df_sr_list = []
    for s in 1:nsims
        df = sim_blockwise_TD0TD1SRMB_subject(0.0, 0.0, βSR, 0.0, βBoat, island_bias, boat_bias, αHome, αAway, αM, γ, s)
        df[!, "model_type"] = "Pure SR"
        df[!, "subject"] = string("SR_", lpad(s, 3, "0"))
        push!(df_sr_list, df)
    end
    df_sr = vcat(df_sr_list...)
    
    # SR-IS
    println("  SR-IS...")
    df_sr_is_list = []
    for s in 1:nsims
        df = sim_blockwise_SRIS_subject(βSR, βBoat, island_bias, boat_bias, αHome, αAway, αM, γ, s)
        df[!, "subject"] = string("SRIS_", lpad(s, 3, "0"))
        push!(df_sr_is_list, df)
    end
    df_sr_is = vcat(df_sr_is_list...)
    
    # Hybrid
    println("  Hybrid...")
    df_hybrid_list = []
    for s in 1:nsims
        df = sim_blockwise_TD0TD1SRMB_subject(0.0, 0.0, βSR/2, βMB/2, βBoat, island_bias, boat_bias, αHome, αAway, αM, γ, s)
        df[!, "model_type"] = "Hybrid"
        df[!, "subject"] = string("HYB_", lpad(s, 3, "0"))
        push!(df_hybrid_list, df)
    end
    df_hybrid = vcat(df_hybrid_list...)
    
    # Add regressors and run analysis
    println("Adding regressors...")
    for df in [df_mb, df_sr, df_sr_is, df_hybrid]
        add_all_regressors!(df)
    end
    
    # Run regressions
    println("Running regressions...")
    results = Dict()
    
    for (name, df) in [("Pure MB", df_mb), ("Pure SR", df_sr), ("SR-IS", df_sr_is), ("Hybrid", df_hybrid)]
        df_reg = filter_df(df)
        
        f = @formula(endBranchLeft ~ 1 + rewardₜ₋₁ * endStateSiblingPriorNoRewardₜ₋₁ * parentPriorMoveToEndStateₜ₋₁ + 
                    (1 + rewardₜ₋₁ + endStateSiblingPriorNoRewardₜ₋₁ + parentPriorMoveToEndStateₜ₋₁ | subject))
        
        m = fit(MixedModel, f, df_reg, Bernoulli())
        results[name] = m
        
        println("\n$name model coefficients:")
        println(coef(m))
    end
    
    return results, df_mb, df_sr, df_sr_is, df_hybrid
end

run_sr_is_analysis (generic function with 1 method)

In [4]:
# 1. Simulation function matching notebook style
function regression_sr_is_blockwise_cov_fixedalpha(centering, biastype, interactiontype, αHome, αAway; nsims=200, seed=nothing)
    suffix = "reg_sr_is_blockwise_cov_n-$(nsims)_aHome-$(αHome)_aAway-$(αAway)"
    if !isnothing(seed)
        Random.seed!(seed)
        suffix *= "_seed-$(seed)"
    end
    
    # Fixed parameters for SR-IS
    βSR = 2.0
    βBoat = 2.0
    island_bias = 0.0
    boat_bias = 0.0
    αM = 0.5
    γ = 0.95
    
    dfs = []
    for s in 1:nsims
        df = sim_blockwise_SRIS_subject(βSR, βBoat, island_bias, boat_bias, αHome, αAway, αM, γ, s)
        df[!, "sub"] .= s
        df[!, "subject"] .= lpad(s, 3, "0")
        push!(dfs, df)
    end
    df_reg = vcat(dfs...)
    
    add_all_regressors!(df_reg)
    df_reg = filter_df(df_reg)
    (extra_suffix, center_fn, fm) = make_reg_suffix(centering, biastype, interactiontype)
    suffix *= extra_suffix
    df_reg = center_fn(df_reg)
    @info suffix
    flush(stdout)
    
    result_reg = fit(MixedModel, fm, df_reg, Binomial(), progress=false)
    
    # Save results
    mkpath(output_dir)
    save("$(output_dir)/$(suffix)_coeftable.jld2", "results", coeftable(result_reg); compress=true)
    
    result_reg
end

regression_sr_is_blockwise_cov_fixedalpha (generic function with 1 method)

In [5]:
# 2. Run the analyses (matching notebook calls)
reg_rwdonly_sim_sr_is = regression_sr_is_blockwise_cov_fixedalpha(:rwdchoice, :rwdonly, nothing, 0.5, 0.5; seed=0)
reg_full_sim_sr_is = regression_sr_is_blockwise_cov_fixedalpha(:rwdchoice, :neighborboatlag5policylag5sameboatlag5, nothing, 0.5, 0.5; seed=0)

println("SR-IS analyses complete!")

UndefVarError: UndefVarError: `SRISTwoStepSoftmax` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [ ]:
# 2. Run the analyses (matching notebook calls)
reg_rwdonly_sim_sr_is = regression_sr_is_blockwise_cov_fixedalpha(:rwdchoice, :rwdonly, nothing, 0.5, 0.5; seed=0)
reg_full_sim_sr_is = regression_sr_is_blockwise_cov_fixedalpha(:rwdchoice, :neighborboatlag5policylag5sameboatlag5, nothing, 0.5, 0.5; seed=0)

println("SR-IS analyses complete!")

# 3. Load and compare with other models (if their results exist)
# This part loads previously saved results from other models
try
    reg_rwdonly_sim_mb = DataFrame(load("$(output_dir)/reg_mb_blockwise_cov_n-200_aHome-0.5_aAway-0.5_seed-0_center-rwd-choice_rwdonly_coeftable.jld2", "results"))
    reg_rwdonly_sim_sr = DataFrame(load("$(output_dir)/reg_sr_blockwise_cov_n-200_aHome-0.5_aAway-0.5_seed-0_center-rwd-choice_rwdonly_coeftable.jld2", "results"))
    reg_full_sim_mb = DataFrame(load("$(output_dir)/reg_mb_blockwise_cov_n-200_aHome-0.5_aAway-0.5_seed-0_center-rwd-choice_neighborboatlag5policylag5sameboatlag5_coeftable.jld2", "results"))
    reg_full_sim_sr = DataFrame(load("$(output_dir)/reg_sr_blockwise_cov_n-200_aHome-0.5_aAway-0.5_seed-0_center-rwd-choice_neighborboatlag5policylag5sameboatlag5_coeftable.jld2", "results"))
    
    println("\nLoaded comparison model results successfully")
catch e
    println("\nCould not load comparison results - run other models first")
end

# 4. Extract coefficients for comparison
rwd_name = "rewardₜ₋₁"
mb_name = "rewardₜ₋₁ & lag1_neighborboat_reg"
sr_name = "rewardₜ₋₁ & lag1_policy_reg"

# Get SR-IS coefficients
coefs_full_sr_is, stderrs_full_sr_is, pvals_full_sr_is, names_full_sr_is = get_coefs(reg_full_sim_sr_is, [rwd_name, mb_name, sr_name])

println("\n" * "="^50)
println("SR-IS Model Results")
println("="^50)
println("\nFull Model Coefficients:")
println("  Reward main effect: $(round(coefs_full_sr_is[1], digits=3)) ± $(round(1.96*stderrs_full_sr_is[1], digits=3))")
println("  MB interaction: $(round(coefs_full_sr_is[2], digits=3)) ± $(round(1.96*stderrs_full_sr_is[2], digits=3))")
println("  SR interaction: $(round(coefs_full_sr_is[3], digits=3)) ± $(round(1.96*stderrs_full_sr_is[3], digits=3))")

# 5. Create Figure 2-style visualization
using Plots

# If you have all model results loaded, create comparison plot
# This matches the style in the notebook
if @isdefined(reg_full_sim_mb) && @isdefined(reg_full_sim_sr)
    # Extract all coefficients
    models = ["MB", "SR", "SR-IS", "Participants"]  # Add Hybrid if you have it
    mb_interactions = []
    sr_interactions = []
    
    # ... (plotting code matching notebook style)
end

println("\nAnalysis complete!")

In [ ]:
# Function matching the notebook style for SR-IS
function regression_sr_is_blockwise_cov_fixedalpha(centering, biastype, interactiontype, αHome, αAway; nsims=200, seed=nothing)
    suffix = "reg_sr_is_blockwise_cov_n-$(nsims)_aHome-$(αHome)_aAway-$(αAway)"
    if !isnothing(seed)
        Random.seed!(seed)
        suffix *= "_seed-$(seed)"
    end
    
    # Parameters similar to the notebook
    # Using fixed parameters instead of loading from fitted model
    βSR = 2.0
    βBoat = 2.0
    island_bias = 0.0
    boat_bias = 0.0
    αM = 0.5
    γ = 0.95
    
    dfs = []
    for s in 1:nsims
        df = sim_blockwise_SRIS_subject(βSR, βBoat, island_bias, boat_bias, αHome, αAway, αM, γ, s)
        df[!, "sub"] .= s
        df[!, "subject"] .= lpad(s, 3, "0")
        push!(dfs, df)
    end
    df_reg = vcat(dfs...)
    
    add_all_regressors!(df_reg)
    df_reg = filter_df(df_reg)
    (extra_suffix, center_fn, fm) = make_reg_suffix(centering, biastype, interactiontype)
    suffix *= extra_suffix
    df_reg = center_fn(df_reg)
    @info suffix
    flush(stdout)
    
    result_reg = fit(MixedModel, fm, df_reg, Binomial(), progress=false)
    save("$(output_dir)/$(suffix)_coeftable.jld2", "results", coeftable(result_reg); compress=true)
    result_reg
end

# Run the same analyses as in the notebook
println("Running SR-IS analyses matching notebook structure...")

# Reward-only model
regression_sr_is_blockwise_cov_fixedalpha(:rwdchoice, :rwdonly, nothing, 0.5, 0.5; seed=0)

# Full model with interactions
regression_sr_is_blockwise_cov_fixedalpha(:rwdchoice, :neighborboatlag5policylag5sameboatlag5, nothing, 0.5, 0.5; seed=0)

# Now run the comparison with other models (matching the notebook)
function run_notebook_style_comparison()
    # Parameters
    nsims = 50
    αHome = 0.5
    αAway = 0.5
    seed = 0
    
    # Run all models with same seed
    Random.seed!(seed)
    
    # Pure MB (from notebook)
    println("Running Pure MB...")
    df_mb = regression_mb_blockwise_cov_fixedalpha(:rwdchoice, :neighborboatlag5policylag5sameboatlag5, nothing, αHome, αAway; nsims=nsims, seed=seed)
    
    # Pure SR (from notebook)
    println("Running Pure SR...")
    df_sr = regression_sr_blockwise_cov_fixedalpha(:rwdchoice, :neighborboatlag5policylag5sameboatlag5, nothing, αHome, αAway; nsims=nsims, seed=seed)
    
    # SR-IS (our new model)
    println("Running SR-IS...")
    df_sr_is = regression_sr_is_blockwise_cov_fixedalpha(:rwdchoice, :neighborboatlag5policylag5sameboatlag5, nothing, αHome, αAway; nsims=nsims, seed=seed)
    
    # Hybrid MB/SR (from notebook)
    println("Running Hybrid...")
    df_hybrid = regression_mb_sr_blockwise_cov_fixedalpha(:rwdchoice, :neighborboatlag5policylag5sameboatlag5, nothing, αHome, αAway; nsims=nsims, seed=seed)
    
    # Participants (from notebook)
    println("Running Participants...")
    df_participants = regression_participants(:rwdchoice, :neighborboatlag5policylag5sameboatlag5, nothing)
    
    return Dict(
        "Pure MB" => df_mb,
        "Pure SR" => df_sr,
        "SR-IS" => df_sr_is,
        "Hybrid" => df_hybrid,
        "Participants" => df_participants
    )
end

# Extract and compare coefficients (matching notebook style)
function extract_key_coefficients(results)
    rwd_name = "rewardₜ₋₁"
    mb_name = "rewardₜ₋₁ & lag1_neighborboat_reg"
    sr_name = "rewardₜ₋₁ & lag1_policy_reg"
    
    coef_summary = Dict()
    
    for (model_name, result) in results
        coefs_rwd, stderrs_rwd, pvals_rwd, names_rwd = get_coefs(result, [rwd_name])
        coefs_mb, stderrs_mb, pvals_mb, names_mb = get_coefs(result, [mb_name])
        coefs_sr, stderrs_sr, pvals_sr, names_sr = get_coefs(result, [sr_name])
        
        coef_summary[model_name] = Dict(
            "reward_main" => coefs_rwd[1],
            "reward_stderr" => stderrs_rwd[1],
            "mb_interaction" => isnothing(coefs_mb) ? 0.0 : coefs_mb[1],
            "mb_stderr" => isnothing(stderrs_mb) ? 0.0 : stderrs_mb[1],
            "sr_interaction" => isnothing(coefs_sr) ? 0.0 : coefs_sr[1],
            "sr_stderr" => isnothing(stderrs_sr) ? 0.0 : stderrs_sr[1]
        )
    end
    
    return coef_summary
end

# Create Figure 2-style plot (matching notebook)
function create_figure2_style_plot(coef_summary)
    models = ["Pure MB", "Pure SR", "SR-IS", "Hybrid", "Participants"]
    
    # Extract values for plotting
    reward_effects = [coef_summary[m]["reward_main"] for m in models]
    mb_interactions = [coef_summary[m]["mb_interaction"] for m in models]
    sr_interactions = [coef_summary[m]["sr_interaction"] for m in models]
    
    # Create subplots matching Figure 2 d-g
    p1 = bar(models, reward_effects,
             title="Main Effect of Reward",
             ylabel="Regression Coefficient",
             ylims=(-0.5, 1.5),
             legend=false,
             color=[:dodgerblue, :crimson, :purple, :forestgreen, :orange])
    
    p2 = bar(models, mb_interactions,
             title="MB Interaction (Neighbor × Reward)",
             ylabel="Regression Coefficient",
             ylims=(-0.5, 1.0),
             legend=false,
             color=[:dodgerblue, :crimson, :purple, :forestgreen, :orange])
    
    p3 = bar(models, sr_interactions,
             title="SR Interaction (Policy × Reward)",
             ylabel="Regression Coefficient",
             ylims=(-0.5, 1.0),
             legend=false,
             color=[:dodgerblue, :crimson, :purple, :forestgreen, :orange])
    
    # Combine plots
    plot(p1, p2, p3, layout=(1,3), size=(1200, 400))
end

# Main execution matching notebook style
println("\n" * "="^50)
println("SR WITH IMPORTANCE SAMPLING ANALYSIS")
println("Notebook-style implementation")
println("="^50 * "\n")

# Run the comparison
results = run_notebook_style_comparison()

# Extract coefficients
coef_summary = extract_key_coefficients(results)

# Print summary
println("\nCoefficient Summary:")
println("-"^50)
for model in ["Pure MB", "Pure SR", "SR-IS", "Hybrid", "Participants"]
    println("\n$model:")
    println("  Reward main effect: $(round(coef_summary[model]["reward_main"], digits=3)) ± $(round(1.96*coef_summary[model]["reward_stderr"], digits=3))")
    println("  MB interaction: $(round(coef_summary[model]["mb_interaction"], digits=3)) ± $(round(1.96*coef_summary[model]["mb_stderr"], digits=3))")
    println("  SR interaction: $(round(coef_summary[model]["sr_interaction"], digits=3)) ± $(round(1.96*coef_summary[model]["sr_stderr"], digits=3))")
end

# Create and save plot
plt = create_figure2_style_plot(coef_summary)
savefig("figures/sr_is_comparison_fig2_style.pdf")

println("\nAnalysis complete! Plot saved to figures/sr_is_comparison_fig2_style.pdf")